In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import mean_squared_error

population_training_df = pd.read_csv('population_training.csv', encoding='cp1252').dropna(axis=0)
population_testing_df = pd.read_csv('population_testing.csv', encoding='cp1252').dropna(axis=0)
print(population_training_df.shape)
print(population_testing_df.shape)

population_training_matrix = population_training_df.values     #numpy array
population_testing_matrix = population_testing_df.values       #numpy array
print(population_training_matrix.shape)
print(population_testing_matrix.shape)


(40, 213)
(17, 213)
(40, 213)
(17, 213)


In [0]:
#function to get minimum value in a dictionary and its corresponding key
def minimum_keys(dic):
    minimum = min(dic.values())
    keys = filter(lambda x:dic[x] == minimum,dic.keys())
    return [i for i in keys],minimum



In [0]:
##Lasso Regularization
from sklearn.linear_model import Lasso
from sklearn import metrics
import statistics

In [0]:
mse_dic={}                                                    #dictionary to track the mse values 
df=pd.DataFrame(index=population_training_df.columns[1:])   # for parameters
#df_pred=pd.DataFrame(population_testing_df['Id'],columns=['Id'])  # for predicition
i=True
j=[]
Alpha=100

lasso_mse=[]
while i: 
  for a in population_training_df.columns[1:]:                             #loop for columns, looping after id column
  #a='Brazil'
    x_train=population_training_df.drop(columns=['Id',a])
    x_test=population_testing_df.drop(columns=['Id',a]).copy()            #made a copy ensuring changeds in x_test doestnt affect population_testing_df
    y_train=population_training_df[a]
    y_test=population_testing_df[a]

    lassoReg = Lasso(alpha=Alpha, normalize=True)

    lassoReg.fit(x_train,y_train)
    i=np.count_nonzero(lassoReg.coef_)>=1    #condition for lasso, we need atleast one predictor
    j.append(np.count_nonzero(lassoReg.coef_))   #creating a list of number of non zero parameters for each country
    #df.loc[~a,a]
    #population_training_df.loc[population_training_df.index!=1,'Aruba']
    df.loc[a,a]=0                                             #obviously this parameter would be 0 as this is the target country
    df.loc[df.index!=a,a]=lassoReg.coef_                      ##LASSO PARAMETER dataframe 
    if np.count_nonzero(lassoReg.coef_)<5:                      # countries(target) for which we have less than 5 predictors 
      y_pred=lassoReg.predict(x_test)
    else:
      for n in x_test.columns:
        if n not in df.nlargest(4, [a])[a].index:             #countries with more than 4 predictors, we choose 4 predictors with the largest parameter(lasso coeffecient)
          #df[a]=0
          df.loc[n,a]=0                                       #setting coeffecients of the remaining(excluding top 4) predictors to zero and storing it in parameters dataframe
          x_test[n]=0                                         #setting the x_test values to 0 for remaining pedictors so that there is no effect of the these predictors on our prediction of the target country
          y_pred=lassoReg.predict(x_test)
    #df_pred[a]=y_pred
    #y_pred=lassoReg.predict(x_test)
        #####for mse

    lasso_mse.append((metrics.mean_squared_error(y_test, y_pred)))    #creating a list of mse for each country for a fixed alpha
  #####

    if i==False:                                                  #this conidtion would break the loop if for a target country there are 0 predictors
           break
  #k=np.array(j)              #this seeems extra
  #if (k==0).sum()>0:
    #break
  mse_dic[Alpha]=statistics.mean(lasso_mse)                         #taking mean value of the mse of all countries for a fixed alpha and storing it in a dictionary
  Alpha+=100


In [0]:
minimum_keys(mse_dic)

([100], 1.310787044400866e+16)

In [0]:
mse_dic

{100: 1.310787044400866e+16, 200: 1.3283994854908906e+16}

In [0]:
minimum_keys(mse_dic)[0][0]

100

In [0]:
the_alpha=150

In [0]:
the_alpha=minimum_keys(mse_dic)[0][0]     ## alpha value giving least mse for predicting at 1999, its 152

In [0]:
#loop for predicting at alpha giving least mse 
mse_dic={}
df=pd.DataFrame(index=population_training_df.columns[1:])   # for parameters
df_pred=pd.DataFrame(population_testing_df['Id'],columns=['Id'])  # for predicition
#i=True
#j=[]
#Alpha=1

lasso_mse=[]
for a in population_training_df.columns[1:]:                             #loop for columns, looping after id column
#a='Brazil'
  x_train=population_training_df.drop(columns=['Id',a])
  x_test=population_testing_df.drop(columns=['Id',a]).copy()
  y_train=population_training_df[a]
  y_test=population_testing_df[a]

  lassoReg = Lasso(alpha=the_alpha, normalize=True)

  lassoReg.fit(x_train,y_train)
  #i=np.count_nonzero(lassoReg.coef_)>=1    #condition for lasso
  #j.append(np.count_nonzero(lassoReg.coef_))   
  #df.loc[~a,a]
  #population_training_df.loc[population_training_df.index!=1,'Aruba']
  df.loc[a,a]=0
  df.loc[df.index!=a,a]=lassoReg.coef_                      ##LASSO PARAMETER dataframe 
  y_pred=lassoReg.predict(x_test)
  # if np.count_nonzero(lassoReg.coef_)<5:
  #   y_pred=lassoReg.predict(x_test)
  # else:
  #   for n in x_test.columns:
  #     if n not in df.nlargest(4, [a])[a].index:
  #       #df[a]=0
  #       df.loc[n,a]=0
  #       x_test[n]=0
  #       y_pred=lassoReg.predict(x_test)
  df_pred[a]=y_pred
    
      #####for mse

  lasso_mse.append((metrics.mean_squared_error(y_test, y_pred)))
#####
mse_dic[the_alpha]=statistics.mean(lasso_mse)


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 127656568653.879, tolerance: 39009904451.87751
  positive)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2305871.301716119, tolerance: 1108848.2719975
  positive)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 9889530846.288363, tolerance: 3218450140.0527606
  positive)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 292382690306.7993,

In [0]:
mse_dic

{150: 59003112867534.82}

In [0]:
# to check the number of non zero coeffecients in the parameters data frame df 
out=pd.DataFrame()
for z in df.columns:                                 ####confirm this 
  #print(z,np.count_nonzero(df[z].values))
  out.loc[z,'val']=np.count_nonzero(df[z].values)

out[out.val<5]

,val
Aruba,4.0
Antigua and Barbuda,2.0
Bermuda,4.0
Curacao,3.0
Cayman Islands,4.0
Dominica,3.0
Estonia,4.0
Gibraltar,3.0
Grenada,1.0
Isle of Man,3.0


In [0]:
out.loc['Aruba',:]

val    5.0
Name: Aruba, dtype: float64

In [0]:
df.to_csv('lasso parameters_153.csv')

In [0]:
df.head()

,Aruba,Afghanistan,Angola,Albania,Andorra,United Arab Emirates,Argentina,Armenia,American Samoa,Antigua and Barbuda,Australia,Austria,Azerbaijan,Burundi,Belgium,Benin,Burkina Faso,Bangladesh,Bulgaria,Bahrain,"Bahamas, The",Bosnia and Herzegovina,Belarus,Belize,Bermuda,Bolivia,Brazil,Barbados,Brunei Darussalam,Bhutan,Botswana,Central African Republic,Canada,Switzerland,Channel Islands,Chile,China,Cote d'Ivoire,Cameroon,"Congo, Dem. Rep.",...,Somalia,South Sudan,Sao Tome and Principe,Suriname,Slovak Republic,Slovenia,Sweden,Swaziland,Seychelles,Syrian Arab Republic,Turks and Caicos Islands,Chad,Togo,Thailand,Tajikistan,Turkmenistan,Timor-Leste,Tonga,Trinidad and Tobago,Tunisia,Turkey,Tuvalu,Tanzania,Uganda,Ukraine,Uruguay,United States,Uzbekistan,St. Vincent and the Grenadines,"Venezuela, RB",British Virgin Islands,Virgin Islands (U.S.),Vietnam,Vanuatu,Samoa,Kosovo,"Yemen, Rep.",South Africa,Zambia,Zimbabwe
Aruba,0.0,273.61398,285.654044,38.398908,1.448457,81.97206,422.865768,24.298853,1.004656,0.482786,209.339666,21.748961,99.154134,96.662285,18.90917,117.558638,176.48768,2139.30965,0.0,12.908874,4.49842,0.0,42.875423,3.615472,0.306135,119.167912,2600.557193,0.956499,6.495133,7.971238,32.804334,59.568449,316.555195,40.396851,0.925294,188.256966,14505.485462,353.488876,267.299311,847.512999,...,154.970868,72.708001,2.036973,4.030152,33.540252,8.595633,32.187241,19.438096,0.834328,309.640902,0.365831,134.865394,87.457881,871.21383,107.334473,77.685595,10.925062,0.597236,11.064584,152.353854,910.702814,0.071508,635.161979,441.986364,150.484439,18.361675,2470.674119,420.160244,0.653441,414.540307,0.333553,1.592126,1236.624719,3.170673,1.303375,27.110617,355.62561,741.653009,191.998977,230.752897
Afghanistan,0.0,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.00000,0.0,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
Angola,0.0,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.00000,0.0,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
Albania,0.0,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.00000,0.0,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
Andorra,0.0,0.00000,0.000000,2.748606,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0000

In [0]:
# kk=[]
# for i in df.columns:
#   kk.append(np.count_nonzero(df[i].values))

# #(np.array(kk)==0).sum()
# #kk

In [0]:
# (np.array(kk)<=4).sum()                          ##does the same as out function 

212

In [0]:
df.nlargest(4, [a])[a].index

Index(['Aruba', 'Isle of Man', 'Tonga', 'Malta'], dtype='object')

In [0]:
np.count_nonzero(lassoReg.coef_)

92

In [0]:
df.head

<bound method NDFrame.head of                    Brazil
Aruba         2312.171479
Afghanistan      0.000000
Angola           0.000000
Albania          0.000000
Andorra          0.000000
...                   ...
Kosovo           0.000000
Yemen, Rep.      0.000000
South Africa     0.000000
Zambia           0.000000
Zimbabwe         0.000000

[212 rows x 1 columns]>

In [0]:
df.nlargest(4, [a])[a]

American Samoa              14.782562
Aruba                       12.641078
Andorra                      6.573717
St. Martin (French part)     5.883519
Name: Zimbabwe, dtype: float64

In [0]:
df.head()

In [0]:
df_pred.to_csv('Lasso_output.csv',index=False)

In [0]:
np.count_nonzero(lassoReg.coef_)

16

In [0]:
# mse_dic={}
# df={}
# i=True
# j=[]
# Alpha=151
# while i:   # alpha loop
#     lasso_mse=[]
#     for a in population_training_df.columns[1:]:                             #loop for columns
#         x_train=population_training_df.iloc[:39].drop(columns=['Id',a])
#         x_val=population_training_df.iloc[39:].drop(columns=['Id',a])
#         y_train=population_training_df.iloc[:39][a]
#         y_val=population_training_df.iloc[39:][a]

#         lassoReg = Lasso(alpha=Alpha, normalize=True)

#         lassoReg.fit(x_train,y_train)
#         i=np.count_nonzero(lassoReg.coef_)>=1    #condition for lasso
#         j.append(np.count_nonzero(lassoReg.coef_))   
#         df[a]=pd.DataFrame(lassoReg.coef_,index=x_train.columns,columns=[a])
#         #####for mse
#         t=pd.DataFrame(lassoReg.coef_,index=x_train.columns,columns=[a]).sort_values(by=a,ascending=False)[:4].index.values
#         for n in x_val.columns:
#             if n not in t:
#                 x_val[n]=0

#         y_pred = lassoReg.predict(x_val)

#         lasso_mse.append((metrics.mean_squared_error(y_val, y_pred)))
#         #####
#         if i==False:
#            break
#     k=np.array(j)
#     if (k==0).sum()>0:
#       break
#     mse_dic[Alpha]=statistics.mean(lasso_mse)
#     Alpha+=1

In [0]:
minimum_keys(mse_dic)    


([152], 1.2272129944641636e+16)

In [0]:
mse_dic.keys()

dict_keys([151, 152, 153])

In [0]:
df['Afghanistan'][df['Afghanistan'].values>0]

,Afghanistan
Aruba,251.450129
Antigua and Barbuda,55.229832
Austria,0.631031
Belgium,0.586958
Czech Republic,0.204525
Germany,0.000084
Guyana,1.750151
Croatia,0.009079
Lebanon,0.429131
Luxembourg,0.427486


In [0]:
mse_dic={}
df=pd.DataFrame(index=x_train.columns)
#for i in range(100*(10**6),1000*(10**6),10**6):   # alpha loop
lasso_mse=[]
for a in population_training_df.columns[1:]:                             #loop for columns
	x_train=population_training_df.drop(columns=['Id',a])
	x_test=population_testing_df.drop(columns=['Id',a])
	y_train=population_training_df[a]
	y_test=population_testing_df[a]

	lassoReg = Lasso(alpha=152, normalize=True)    #the_alpha=152

	lassoReg.fit(x_train,y_train)


	df[a]=lassoReg.coef_
	
  t=pd.DataFrame(lassoReg.coef_,index=x_train.columns,columns=[a]).sort_values(by=a,ascending=False)[:4].index.values
	for n in x_val.columns:
		if n not in t:
			x_val[n]=0

	y_pred = lassoReg.predict(x_val)

	lasso_mse.append((metrics.mean_squared_error(y_val, y_pred)))

mse_dic[the_alpha]=statistics.mean(lasso_mse)

In [0]:
np.count_nonzero(lassoReg.coef_)

0

In [0]:
out=pd.DataFrame()
for z in df.keys():
  #print(z,np.count_nonzero(df[z].values))
  out.loc[z,'val']=np.count_nonzero(df[z].values)

In [0]:
out[out.val<5]

,val
Antigua and Barbuda,2.0
Bermuda,4.0
Curacao,3.0
Cayman Islands,4.0
Dominica,3.0
Estonia,4.0
Gibraltar,2.0
Grenada,1.0
Isle of Man,3.0
St. Kitts and Nevis,3.0


In [0]:
population_testing_df.head()

,Id,Aruba,Afghanistan,Angola,Albania,Andorra,United Arab Emirates,Argentina,Armenia,American Samoa,Antigua and Barbuda,Australia,Austria,Azerbaijan,Burundi,Belgium,Benin,Burkina Faso,Bangladesh,Bulgaria,Bahrain,"Bahamas, The",Bosnia and Herzegovina,Belarus,Belize,Bermuda,Bolivia,Brazil,Barbados,Brunei Darussalam,Bhutan,Botswana,Central African Republic,Canada,Switzerland,Channel Islands,Chile,China,Cote d'Ivoire,Cameroon,...,Somalia,South Sudan,Sao Tome and Principe,Suriname,Slovak Republic,Slovenia,Sweden,Swaziland,Seychelles,Syrian Arab Republic,Turks and Caicos Islands,Chad,Togo,Thailand,Tajikistan,Turkmenistan,Timor-Leste,Tonga,Trinidad and Tobago,Tunisia,Turkey,Tuvalu,Tanzania,Uganda,Ukraine,Uruguay,United States,Uzbekistan,St. Vincent and the Grenadines,"Venezuela, RB",British Virgin Islands,Virgin Islands (U.S.),Vietnam,Vanuatu,Samoa,Kosovo,"Yemen, Rep.",South Africa,Zambia,Zimbabwe
0,2000,90853,20093756,16440924,3089027,65390,3154925,37057452,3069588,57521,83584,19153000,8011566,8048600,6400706,10251250,6865951,11607942,131581243,8170172,664614,297890,3766706,9979610,247315,61833,8339512,175287587,269847,333241,573416,1728340,3754986,30769700,7184250,148725,15262754,1262645000,16686561,15274234,...,9011479,6700656,138606,472390,5388720,1988925,8872109,1061468,81131,16410848,18873,8342559,4970367,62958021,6216205,4516131,871607,98082,1267984,9699197,63240121,9420,34178042,24039274,49175848,3321245,282162411,24650400,107898,24488340,20645,108642,80285562,185063,174610,1700000,17874725,45728315,10531221,12222251
1,2001,92898,20966463,16983266,3060173,67341,3326032,37471509,3050655,58175,85057,19413000,8042293,8111200,6555829,10286570,7076733,11944587,134107160,8009142,697549,303135,3771284,9928549,254984,62504,8496375,177750670,270685,340117,589600,1754935,3832203,31081900,7229854,149793,15444969,1271850000,17040152,15671927,...,9290823,6974442,141622,477740,5378867,1992060,8895960,1072927,81202,16766899,20185,8663012,5111770,63543322,6327125,4564080,892531,98611,1272380,9785701,64191474,9512,35117019,24854892,48683865,3327103,284968955,24964450,107988,24948476,21085,108549,81139919,189290,175566,1701154,18390135,46385006,10824125,12366165
2,2002,94992,21979923,17572649,3051010,70049,3507232,37889370,3033897,58731,86266,19651400,8081957,8171950,6741569,10332785,7295394,12293100,136600667,7837161,735148,309157,3775807,9865548,262206,62912,8653345,180151021,271478,346867,606399,1779953,3907612,31362000,7284753,150901,15623635,1280400000,17366517,16084886,...,9564167,7237276,144889,483044,5376912,1994530,8924958,1080930,83723,17087901,21742,9001689,5251472,64073164,6447688,4610002,923825,99184,1277837,9864326,65143054,9635,36105808,25718048,48202500,3327773,287625193,25271850,108146,25408700,21529,108510,81956496,193956,176582,1702310,18919179,47026173,11120409,12500525
3,2003,97017,23064851,18203369,3039616,73182,3741932,38309379,3017806,59117,87293,19895400,8121423,8234100,6953113,10376133,7520555,12654621,139019001,7775327,778711,315746,3779247,9796749,269130,63325,8810420,182482149,272258,353389,623434,1804339,3981665,31676000,7339001,152038,15799542,1288400000,17679355,16513822,...,9836397,7501642,148372,488332,5373374,1995733,8958229,1087392,82781,17415266,23410,9353201,5391401,64554952,6576877,4655741,960852,99789,1284052,9939678,66085803,9767,37149072,26624820,47812950,3325637,290107933,25567650,108350,25868523,22000,108506,82747662,198964,177662,1703466,19462086,47648727,11421984,12633897
4,2004,98737,24118979,18865716,3026939,76244,4087931,38728696,3000612,59264,88257,20127400,8171966,8306500,7182451,10421137,7750004,13030569,141307489,7716860,829848,322526,3781287,9730146,276089,63740,8967741,184738458,273091,359523,640282,1829330,4055036,31995000,7389625,153170,15973778,1296075000,17997738,16959081,...,10116228,7787655,151969,493630,5372280,1997012,8993531,1095053,82475,17806638,25028,9710043,5534598,65002231,6712841,4703398,996698,100406,1290535,10017601,67007855,9894,38249984,27568436,47451600,3324096,292805298

In [0]:
df['Isle of Man'][df['Isle of Man'].values!=0]

,Isle of Man
Djibouti,0.012004
Sweden,0.006915
Virgin Islands (U.S.),0.085503


In [0]:
minimum_keys(mse_dic)
#np.count_nonzero(lassoReg.coef_)

([31], 1.4389380583983402e+16)

In [0]:
mse_dic.keys()

dict_keys([1, 6, 11, 16, 21, 26, 31, 36])

In [0]:
np.count_nonzero(lassoReg.coef_)

3

In [0]:
df['Isle of Man'][df['Isle of Man'].values!=0]

,Isle of Man
Djibouti,0.012004
Sweden,0.006915
Virgin Islands (U.S.),0.085503


In [0]:
k=[]
for m in range(0,10):
  k.append(m)
j=np.array(k)
j==0

array([ True, False, False, False, False, False, False, False, False,
       False])

In [0]:
(j==0).sum()

1

In [0]:
if True:
  print(2+2)

4


In [0]:
pd.DataFrame(lassoReg.coef_,index=x_train.columns,columns=[a]).sort_values(by=a,ascending=False)

,Antigua and Barbuda
Gibraltar,1.765196
Afghanistan,0.000677
Mauritius,0.000000
Malaysia,0.000000
Namibia,0.000000
...,...
Greece,0.000000
Grenada,-0.000000
Greenland,0.000000
Guatemala,0.000000


In [0]:
minimum_keys(mse_dic)

([7000], 1856993164120708.0)

In [0]:
pd.DataFrame(lassoReg.coef_,index=x_train.columns,columns=[a]).sort_values(by=a,ascending=False)

,Zimbabwe
Botswana,0.761886
Algeria,0.153720
"Iran, Islamic Rep.",0.015567
Nauru,0.000000
Malaysia,0.000000
...,...
Equatorial Guinea,0.000000
Greece,0.000000
Grenada,0.000000
Greenland,0.000000


In [0]:
out=pd.DataFrame()
for z in df.keys():
  #print(z,np.count_nonzero(df[z].values))
  out.loc[z,'val']=np.count_nonzero(df[z].values)

In [0]:
out[out.val<=4]

,val
Curacao,4.0
Cayman Islands,4.0
Dominica,4.0
Grenada,4.0
Isle of Man,3.0


In [0]:
df['Grenada'][df['Grenada'].values<0]

,Grenada
Afghanistan,-0.000230
Isle of Man,-0.018406


,Isle of Man
Aruba,NaN
Afghanistan,NaN
Angola,NaN
Albania,NaN
Andorra,NaN
...,...
Kosovo,NaN
"Yemen, Rep.",NaN
South Africa,NaN
Zambia,NaN


In [0]:
pd.DataFrame(lassoReg.coef_,index=x_train.columns,columns=[a]).sort_values(by=a,ascending=False)#[:4].index.values

,Zimbabwe
Andorra,15.441744
Marshall Islands,7.710959
St. Martin (French part),7.266686
American Samoa,6.878197
Monaco,2.388262
...,...
"Hong Kong SAR, China",0.000000
Honduras,0.000000
Croatia,0.000000
Haiti,0.000000


In [0]:
minimum_keys(mse_dic)

([10000000], 6230564599052961.0)

In [0]:
mse_dic

{10000000: 6230564599052961.0}

In [0]:
i=np.count_nonzero(lassoReg.coef_)==4
i

False

In [0]:
np.count_nonzero(lassoReg.coef_)<=4 

False

In [0]:
df=pd.DataFrame(index=x_train.columns)
df.head()

""
Aruba
Afghanistan
Angola
Albania
Andorra


In [0]:
for i in x_train.columns:
    df[i]=1

In [0]:
df

,Aruba,Afghanistan,Angola,Albania,Andorra,United Arab Emirates,Argentina,Armenia,American Samoa,Antigua and Barbuda,Australia,Austria,Azerbaijan,Burundi,Belgium,Benin,Burkina Faso,Bangladesh,Bulgaria,Bahrain,"Bahamas, The",Bosnia and Herzegovina,Belarus,Belize,Bermuda,Bolivia,Brazil,Barbados,Brunei Darussalam,Bhutan,Botswana,Central African Republic,Canada,Switzerland,Channel Islands,Chile,China,Cote d'Ivoire,Cameroon,"Congo, Dem. Rep.",...,San Marino,Somalia,South Sudan,Sao Tome and Principe,Suriname,Slovak Republic,Slovenia,Sweden,Swaziland,Seychelles,Syrian Arab Republic,Turks and Caicos Islands,Chad,Togo,Thailand,Tajikistan,Turkmenistan,Timor-Leste,Tonga,Trinidad and Tobago,Tunisia,Turkey,Tanzania,Uganda,Ukraine,Uruguay,United States,Uzbekistan,St. Vincent and the Grenadines,"Venezuela, RB",British Virgin Islands,Virgin Islands (U.S.),Vietnam,Vanuatu,Samoa,Kosovo,"Yemen, Rep.",South Africa,Zambia,Zimbabwe
Aruba,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
Afghanistan,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
Angola,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
Albania,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
Andorra,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Kosovo,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
"Yemen, Rep.",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
South Africa,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
Zambia,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [0]:
al=6
i=10
while al-i!=6:
  print(i)
  i -= 1

10
9
8
7
6
5
4
3
2
1
